In [1]:
import pandas as pd
from operator import itemgetter
import csv
import numpy as np
df = pd.read_csv (r"../data/raw_labelbox_exports/fall2020-april20-rawexport.csv")
df.head()

,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label
0,ck18d6trtvxee0a468wskapqf,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...","{""leadership_spectrum"":""0""}",dcottle@g.clemson.edu,Career - Fall 2019 All Labels,1569965947000,1570196613000,9.550,0_0,1.0,-1,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
1,ck18nc7o2oo7j0944wgthijdz,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...","{""leadership_spectrum"":""0""}",dnsmith@g.clemson.edu,Career - Fall 2019 All Labels,1569982995000,1570196613000,5.928,0_0,1.0,-1,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
2,ck18p4z57mwhv0863z7wgkabn,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...","{""leadership_spectrum"":""0""}",mkollur@g.clemson.edu,Career - Fall 2019 All Labels,1569986016000,1570196613000,5.205,0_0,1.0,-1,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
3,ck1c6isc9q65j0794ol1c2fi5,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...","{""leadership_spectrum"":""0""}",ajames8@g.clemson.edu,Career - Fall 2019 All Labels,1570196613000,1570196613000,6.152,0_0,1.0,-1,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
4,ck1smh34trxe00721kj2w2q00,ck0zz58quomb50bo4d16bfisw,3 - Dependable and ability to work independent...,"{""leadership_spectrum"":""0""}",mkollur@g.clemson.edu,Career - Fall 2019 All Labels,1571190866000,1571936155000,10.098,0_1,1.0,-1,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...


### Replace data function

In [2]:
labelers_dictionary = {
    "ajames8@g.clemson.edu": "a",
    "dnsmith@g.clemson.edu": "b",
    "dcottle@g.clemson.edu": "c",
    "mkollur@g.clemson.edu": "d",
    "jcrosb3@g.clemson.edu": "e"
}

def replace_data(dataframe):
    df = dataframe
    df = df.reset_index() 
    
    for index, row in df.iterrows():
        if row["Dataset Name"] == "mentor_responses_sentences_up_to_f19.csv":
            df.iloc[index, 15] = "Mentor"
        if row["Dataset Name"] == "student_responses_sentences_up_to_f19.csv":
            df.iloc[index, 15] = "Student"
        
        if row["Label"] == "{\"leadership_spectrum\":\"3\"}":
            df.iloc[index, 4] = "IDK"
        if row["Label"] == "{\"leadership_spectrum\":\"2\"}":
            df.iloc[index, 4] = "STC"
        if row["Label"] == "{\"leadership_spectrum\":\"1\"}":
            df.iloc[index, 4] = "TC"
        if row["Label"] == "{\"leadership_spectrum\":\"0\"}":
            df.iloc[index, 4] = "NA"
        if row["Label"] == "{\"leadership_spectrum\":\"4\"}":
            df.iloc[index, 4] = "STF"
        if row["Label"] == "{\"leadership_spectrum\":\"5\"}":
            df.iloc[index, 4] = "TF"
        
        df.iloc[index, 5] = labelers_dictionary[row["Created By"]]
        
    return df; 

In [3]:
df = replace_data(df)
df.groupby("Created By").agg("count")

,index,ID,DataRow ID,Labeled Data,Label,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label
Created By,,,,,,,,,,,,,,,,,
a,971,971,971,971,971,971,971,971,971,971,971,971,0,0,971,971,971
b,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,3887,0,0,3887,3887,3887
c,2693,2693,2693,2693,2693,2693,2693,2693,2693,2693,2693,2693,0,0,2693,2693,2693
d,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,3205,0,0,3205,3205,3205
e,968,968,968,967,968,968,968,968,968,968,968,968,0,0,968,968,968


### Assign final labels

In [4]:
#Function that takes in a a csv file name which is the data frame
#Assigns the Final labels 
def final_label(dataframe, labeler_to_exclude):
    df_drop = None
    # drop labels which were labeled by the labeler_to_exclude
    if labeler_to_exclude != "":
        df_drop = dataframe[dataframe["Created By"] != labeler_to_exclude]
    else:
        df_drop = dataframe
    df_drop.sort_values(by=['DataRow ID'])


    # indices of labels: 0 = NA, 1 = TC, 2 = STC, 3 = IDK, 4= STF, 5 = TF
    styles = {
        "IDK" : 0,
        "NA" : 1,
        "STC": 2,
        "STF": 3,
        "TC" : 4,
        "TF":5
    }

    # labels[0] is the label, label[1] is the count, label[2] is the most authoritative selection
    labels = [["IDK", 0, "z"],["NA", 0, "z"],["STC", 0, "z"],["STF", 0, "z"],["TC", 0, "z"],["TF", 0, "z"]] 
    next_id = ""
    #add/clear column for final label
    df_drop["Final Label"] = ""

    for i in range(len(df_drop.index)):
        label = df_drop.iloc[i, 4]
        person = df_drop.iloc[i, 5]
        curr_id = df_drop.iloc[i, 2]
        next_id = "" if i >= len(df_drop.index)-1 else df_drop.iloc[i+1, 2]
        agreement = df_drop.iloc[i,11]


        if label == "Skip" or label == "{}":
            continue
        else:
            labels[styles[label]][1] += 1

        #keeps track of the "lowest" person, to find most authoritative
        if person < labels[styles[label]][2] or labels[styles[label]][2] == "z":
            labels[styles[label]][2] = person


        if curr_id != next_id :
            labels.sort(key=itemgetter(1), reverse=True)
            if(labels[0][1] == labels[1][1]):

                labels.sort(key = itemgetter(2))


            df_drop.iloc[i,18] = labels[0][0]         

            for j in labels:
                j[1] = 0
                j[2] = "z"
            #the below works because this whole cell of code depends on the sorting IDK < NA < TC < TF
            labels.sort(key=itemgetter(0))
            
    # Remove quotes in Labeled Data and replace empty strings in Final Label with NaN
    # Empty strings are duplicates since the Final Label is only assigned to last row in the group
    df_drop.loc[:,"Labeled Data"] = df_drop.loc[:,"Labeled Data"].str.replace('"', "")
    df_drop["Final Label"].replace("", np.nan, inplace=True)

    # Drop rows with NaN as Final Label (the duplicates)
    # Also drop duplcates in Labeled Data
    df_drop = df_drop.dropna(subset=['Final Label'], axis=0).reset_index(drop=True)
    df_drop = df_drop.drop_duplicates(subset=['Labeled Data'],keep = 'first')
    
    return df_drop

In [5]:
df_final_label = final_label (df, "")
df_final_label.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label,Final Label
0,3,ck1c6isc9q65j0794ol1c2fi5,ck0zz58quomb40bo43bp5bc0g,1 - Ability to learn new technology quickly 2 ...,NA,a,Career - Fall 2019 All Labels,1570196613000,1570196613000,6.152,0_0,1.000,-1,NaN,NaN,Mentor,[],https://image-segmentation-v4.labelbox.com?pro...,NA
1,6,ck24y77de8mdj0757mzj2gzg4,ck0zz58quomb50bo4d16bfisw,3 - Dependable and ability to work independent...,NA,b,Career - Fall 2019 All Labels,1571936154000,1571936155000,13.708,0_1,1.000,-1,NaN,NaN,Mentor,[],https://image-segmentation-v4.labelbox.com?pro...,NA
2,9,ck2x9t77sweoc0748mgnpmmkl,ck0zz58quomb60bo4ghezdmi7,1) Daniel clearly has skills and knowledge tha...,STF,c,Career - Fall 2019 All Labels,1573648709000,1573648709000,17.991,1_0,0.500,-1,NaN,NaN,Mentor,[],https://image-segmentation-v4.labelbox.com?pro...,STF
3,13,ck1c6phvyrr7s0838fs3w7m3c,ck0zz58quomb70bo42ip39a2f,"He's only been here 3 months, but appears to b...",NA,a,Career - Fall 2019 All Labels,1570196926000,1570196926000,3.083,1_1,0.333,-1,NaN,NaN,Mentor,[],https://image-segmentation-v4.labelbox.com?pro...,NA
4,16,ck2d9dydij3c30721sf0q6w0x,ck0zz58quomb80bo4f29i74f9,Develop that skill and help to bring others al...,NA,b,Career - Fall 2019 All Labels,1572438675000,1572438675000,2.467,1_2,1.000,-1,NaN,NaN,Mentor,[],https://image-segmentation-v4.labelbox.com?pro...,NA


## Write dataframe to csv

In [7]:
df_clean = df_final_label.loc[:,['Labeled Data','Final Label', 'Dataset Name']]
df_clean.to_csv(r"../data/data_for_model/April-20-F2020-CLEAN.csv", index=False, quoting = csv.QUOTE_NONNUMERIC)

## Calculate F1-score of each labeler to set a benchmark for model
*Requires df_clean to be created*

In [113]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import label_binarize
list_of_labelers = ["ajames8@g.clemson.edu", "dnsmith@g.clemson.edu", "dcottle@g.clemson.edu", "mkollur@g.clemson.edu", "jcrosb3@g.clemson.edu"]
threeCat = {
    'NA': 'NA',
    'TC': 'TC',
    'STC': 'TC',
    'IDK': 'NA',
    'STF': 'TF',
    'TF': 'TF'
}

for i in list_of_labelers:
    # get final labels excluding that labeler
    df_exclude = final_label(df, labelers_dictionary[i])
    # join dataframes based on the DataRow ID, which is all the Labeled Data the 2 have in common
    df_join = pd.merge(df, df_exclude, on='DataRow ID', how='inner')
    # drop any rows which weren't labeled by that labeler
    df_join = df_join[df_join['Created By_x'] == labelers_dictionary[i]]
    # drop any rows where final label is Skip
    df_join = df_join[df_join['Label_x'] != 'Skip']
    df_join = df_join[df_join['Final Label_y'] != 'Skip']
    
    y_true = df_join['Final Label_y']
    y_pred = df_join['Label_x']

    classes = ['NA', 'TC', 'STC', 'IDK', 'STF', 'TF']
    y_true_bin = label_binarize(y_true, classes)
    y_pred_bin = label_binarize(y_pred, classes)

    print("Classification Report Six Categories for labeler " + i)
    print(classification_report(y_true_bin, y_pred_bin))
    
    # now do 3 categories
    df_join['Final Label_y'] = df_join['Final Label_y'].apply(lambda x: threeCat[x])
    df_join['Label_x'] = df_join['Label_x'].apply(lambda x: threeCat[x])
    
    y_true = df_join['Final Label_y']
    y_pred = df_join['Label_x']

    classes = ['NA', 'TC', 'TF']
    y_true_bin = label_binarize(y_true, classes)
    y_pred_bin = label_binarize(y_pred, classes)

    print("Classification Report Three Categories for labeler " + i)
    print(classification_report(y_true_bin, y_pred_bin))

/home/dantheman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

Classification Report Six Categories for labeler ajames8@g.clemson.edu
              precision    recall  f1-score   support

           0       0.80      0.90      0.85       660
           1       1.00      0.09      0.17        33
           2       0.19      0.23      0.21        30
           3       0.22      0.17      0.19       101
           4       0.32      0.31      0.32        77
           5       0.43      0.16      0.23        56

   micro avg       0.69      0.69      0.69       957
   macro avg       0.49      0.31      0.33       957
weighted avg       0.67      0.69      0.66       957
 samples avg       0.69      0.69      0.69       957

Classification Report Three Categories for labeler ajames8@g.clemson.edu
              precision    recall  f1-score   support

           0       0.86      0.93      0.89       761
           1       0.38      0.24      0.29        63
           2       0.57      0.41      0.47       133

   micro avg       0.81      0.81      0.

/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Classification Report Six Categories for labeler dnsmith@g.clemson.edu
              precision    recall  f1-score   support

           0       0.90      0.67      0.77      1585
           1       0.33      0.37      0.34       123
           2       0.21      0.21      0.21       168
           3       0.11      0.31      0.16       115
           4       0.26      0.40      0.31       182
           5       0.30      0.50      0.38       112

   micro avg       0.57      0.57      0.57      2285
   macro avg       0.35      0.41      0.36      2285
weighted avg       0.70      0.57      0.62      2285
 samples avg       0.57      0.57      0.57      2285

Classification Report Three Categories for labeler dnsmith@g.clemson.edu
              precision    recall  f1-score   support

           0       0.87      0.78      0.82      1700
           1       0.45      0.47      0.46       291
           2       0.41      0.65      0.50       294

   micro avg       0.72      0.72      0.

/home/dantheman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

...

/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/dantheman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

Classification Report Six Categories for labeler mkollur@g.clemson.edu
              precision    recall  f1-score   support

           0       0.72      0.87      0.79      1263
           1       0.40      0.20      0.27       103
           2       0.20      0.22      0.21       146
           3       0.16      0.08      0.11       274
           4       0.31      0.27      0.29       225
           5       0.40      0.17      0.23       139

   micro avg       0.59      0.59      0.59      2150
   macro avg       0.37      0.30      0.32      2150
weighted avg       0.53      0.59      0.55      2150
 samples avg       0.59      0.59      0.59      2150

Classification Report Three Categories for labeler mkollur@g.clemson.edu
              precision    recall  f1-score   support

           0       0.80      0.88      0.84      1537
           1       0.43      0.37      0.39       249
           2       0.55      0.38      0.45       364

   micro avg       0.74      0.74      0.

/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Classification Report Six Categories for labeler jcrosb3@g.clemson.edu
              precision    recall  f1-score   support

           0       0.69      0.64      0.66       152
           1       0.31      0.62      0.42        26
           2       0.31      0.18      0.23        44
           3       0.21      0.05      0.08        62
           4       0.28      0.42      0.34        55
           5       0.25      0.45      0.32        31

   micro avg       0.44      0.44      0.44       370
   macro avg       0.34      0.39      0.34       370
weighted avg       0.44      0.44      0.42       370
 samples avg       0.44      0.44      0.44       370

Classification Report Three Categories for labeler jcrosb3@g.clemson.edu
              precision    recall  f1-score   support

           0       0.79      0.57      0.66       214
           1       0.58      0.64      0.61        70
           2       0.47      0.76      0.58        86

   micro avg       0.62      0.62      0.

/home/dantheman/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
